# 4th Session: Parallel Processes with Semaphores and Memory Locks

In this session we will implements a parallel proceses with sempahores and memory locks

In [1]:
%pylab inline
import multiprocessing as mp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from multiprocessing import Semaphore, Lock, Process, Pool
import ctypes
import myfunctions2 as my
import time

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In the next cells, we import the importlib module, needed to reload tue updated myfunctions2 module
evry time we change the functions

In [2]:
import importlib

In [3]:
importlib.reload(my)

<module 'myfunctions2' from '/home/yamroza/Documents/SEMV/MassiveComputing/Session_7/myfunctions2.py'>

This first parallel Multiple Instructions, Multiple Data (MIMD) parallel program will try to make deposits and withdraw over a shared memory value, called balance. 
It will start with value of 100.

The use of Pool execution is oriented to Single Instruction, Multiple Data (SIMD) parallel processes: We define just one execution function, which will be applied to multiple data.

Now, if we will execute a Multiple Instruction, Multiple Data (MIMD) program, we need to use the *multiprocessing.Process* class.

In this case, we construct as many of multiprocessing.Process objects as we need, assigning which function will be executed in each one of the parallel processes, and passing their arguments values

After defining the Process objects, we start those processes with the method start() and each one of them will start to run.

To synchronize the execution, we can wait until one parallel process ends using the method join()



In [4]:
# Python program to illustrate  
# the concept of race condition 
# in multiprocessing 
 
  
# function to withdraw from account 

  
def perform_transactions(): 
  
    # initial balance (in shared memory) 
    balance = mp.Value('i', 100) 
  
    # creating new processes 
    p1 = mp.Process(target=my.withdraw, args=(balance,)) 
    p2 = mp.Process(target=my.deposit, args=(balance,)) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # print final balance 
    print("Final balance = {}".format(balance.value)) 

We will run 10 times the main function "perform_transactions".
Check 2 things: final value and time used

In [5]:
start_time=time.time()
for _ in range(10): 
    # perform same transaction process 10 times 
    perform_transactions()
print("--- %s seconds ---" % (time.time() - start_time))


Final balance = 365
Final balance = -636
Final balance = -363
Final balance = 275
Final balance = 330
Final balance = 77
Final balance = 138
Final balance = 911
Final balance = 134
Final balance = -271
--- 0.5092284679412842 seconds ---


Here we have a race condition, making several hazzards. In this case, we says: we have a race condition

We do not controls the state of the shared variable *balance*
Check the execution time

### Avoid Race conditions
In order to avoid race conditions, we have to use locks.

We can use the inner lock properties in the Value object o use a global lock object

Now, we create, more or less the same code, using locks:

In [6]:
def perform_transactions2(): 
  
    # initial balance (in shared memory) 
    balance = mp.Value('i', 100) 
  
    # creating a lock object 
    lock = mp.Lock() 
  
    # creating new processes 
    p1 = mp.Process(target=my.withdraw2, args=(balance,lock)) 
    p2 = mp.Process(target=my.deposit2, args=(balance,lock)) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # print final balance 
    print("Final balance = {}".format(balance.value)) 

Check, again, the final balance value and the execution time.

In [7]:
start_time=time.time()
for _ in range(10): 
    # perform same transaction process 10 times 
    perform_transactions2() 
print("--- %s seconds ---" % (time.time() - start_time))

Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
--- 0.680781364440918 seconds ---


## Practical Work:

Using your previous coded programs, create a program which applies tow filters to an image, using an edge filter, two kinds of filters, dirst and second order, and writing it in a two shared memory allocations.

Synchronize it to:
* Avoid any memory hazzard
* Wait until both filtering process end

After ends both filtering processes, whe have to read the memory results and compare pixel to pixel both filtered images and select the maximum, to select where the borders were detected.

Both filters are the following:

In [ ]:
#This is the this is the diferential first order differential edge filter sobel operator
my_filter1=np.zeros((3,3))
my_filter1[0,0]=1
my_filter1[1,0]=2
my_filter1[2,0]=1
my_filter1[0,2]=-1
my_filter1[1,2]=-2
my_filter1[2,2]=-1


In [ ]:
#This is the this is the diferential first order differential edge filter sobel operator
my_filter2=np.zeros((3,3))
my_filter2[0,0]=1
my_filter2[0,1]=2
my_filter2[0,2]=1
my_filter2[2,0]=-1
my_filter2[2,1]=-2
my_filter2[2,2]=-1

In [ ]:
image= np.array(Image.open('cat2.jpg'))

In [ ]:
print(image.dtype)

In [ ]:
print(image.shape)

In [ ]:
plt.figure()
plt.imshow(image)

In [ ]:
data_buffer_size=image.shape[0]*image.shape[1]*image.shape[2]
print(data_buffer_size)

In [ ]:
shared_space1= Array(ctypes.c_byte,data_buffer_size)

In [ ]:
shared_data1=my.tonumpyarray(shared_space1)
result_matrix1=shared_data.reshape(image.shape)

In [ ]:
shared_space2= Array(ctypes.c_byte,data_buffer_size)

In [ ]:
shared_data2=my.tonumpyarray(shared_space2)
result_matrix2=shared_data2.reshape(image.shape)

In [ ]:
def parallel_process(image,shared_space_1, shared_space_2):
    #takes the number of processors and divide by 2, for fair play
    numprocess = int(mp.cpu_count()/2)
    #creates a lock instance
    lock = mp.Lock()
    #defines both processes
    p1 = mp.Process(target=my.parallel_filter1, args=(image,shared_space1,my_filter1,numprocess,lock)) 
    p2 = mp.Process(target=my.parallel_filter2, args=(image,shared_space2,my_filter1,numprocess,lock))
    #fires both processes in parallel
    p1.start()
    p2.start()
    
    #Now, whe have to wait until both parallel tasks
    #<fill code here> to wait both processes end before continues
    
    p3 = mp.Process(target=my.parallel_postprocess, args=(image,shared_space1,shared_space2,numprocesslock))
    
    #<fill code here tho fire the third process and wait until it ends>
    

In [ ]:
parallel_process(image,shared_data1,shared_data2)